In [ ]:
import globus_automate_client
import mdf_toolbox
import json

In [ ]:
native_app_id = "417301b1-5101-456a-8a27-423e71a2ae26"  # Premade native app ID (from CFDE)
flows_client = globus_automate_client.create_flows_client(native_app_id)

In [ ]:
from getpass import getpass
aws_id = getpass("AWS ID: ")
aws_secret = getpass("AWS Secret: ")
aws_region = "us-east-1"

In [ ]:
from getpass import getpass
smtp_user = getpass("SMTP Username: ")
smtp_pass = getpass("SMTP Password: ")
smtp_hostname = "email-smtp.us-east-1.amazonaws.com"

In [ ]:
# Definition
loop_flow_def = {
    "title": "#name# Action Loop Flow",
    "description": "Perform multiple #name# Actions",
    "visible_to": ["urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1"],
    "runnable_by": ["urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1"],
    #"visible_to": ["public"],
    #"runnable_by": ["public"],
    "definition": {
        "StartAt": "MoveInputs",
        "States": {
            "MoveInputs": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/expression_eval",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "expressions": [{
                        "expression": "action_inputs",
                        "arguments": {
                            "action_inputs.$": "$.action_inputs"
                        },
                        "result_path": "action_inputs"
                    }]
                },
                "ResultPath": "$.inputs",
                "WaitTime": 86400,
                "Next": "CheckInputs"
            },
            # Supports zero Actions
#            "CheckInputs": {
#                "Type": "Pass",
#                "Result": {
#                    "loop_finished.=": "`$.action_inputs` == []"
                    # Can't guarantee simpleeval allows list creation,
                    # other option is to compare to always-empty list slice
                    #"loop_finished.=": "`$.action_inputs` == `$.action_inputs`[0:0]"
#                },
#                "ResultPath": "$.loop_finished",
#                "Next": "ChooseLoop"
#            },
            "CheckInputs": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/expression_eval",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "expressions": [{
                        #"expression": "action_inputs == []",
                        "expression": "action_inputs == action_inputs[0:0]",
                        "arguments": {
                            "action_inputs.$": "$.inputs.details.action_inputs"
                        },
                        "result_path": "loop_finished"
                    }]
                },
                "ResultPath": "$.loop_info",
                "WaitTime": 86400,
                "Next": "ChooseLoop"
            },
            "ChooseLoop": {
                "Type": "Choice",
                "Choices": [{
                    "Variable": "$.loop_info.details.loop_finished",
                    "BooleanEquals": True,
                    "Next": "FinishLoop"
                }],
                "Default": "#nospace_name#Action"
            }, 
            "#nospace_name#Action": {
                "Type": "Action",
                "ActionUrl": "#action_url#",
                "ActionScope": "#action_scope#",
                "ExceptionOnActionFailure": True,
                "InputPath": "$.inputs.details.action_inputs[0]",
                "ResultPath": "$.#nospace_name#Result",
                "WaitTime": 86400,
                "Next": "RemoveInput"
            },
#            "RemoveInput": {
#                "Type": "Pass",
#                "Result": {
#                    "action_inputs.=": "`$.action_inputs`[1:]"
#                },
#                "ResultPath": "$",
#                "Next": "CheckInputs"
#            },
            "RemoveInput": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/expression_eval",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "expressions": [{
                        "expression": "action_inputs[1:]",
                        "arguments": {
                            "action_inputs.$": "$.inputs.details.action_inputs"
                        },
                        "result_path": "action_inputs"
                    }]
                },
                "ResultPath": "$.inputs",
                "WaitTime": 86400,
                "Next": "CheckInputs"
            },
            "FinishLoop": {
                "Type": "Pass",
                "End": True
            }
        }
    }
}
# Variables encased in ## for formatting later.
# Can't .format() a JSON string, because objects {} are interpreted as
# placeholders. Workaround: Replace #variable# directly.
variables = ["name", "nospace_name", "action_url", "action_scope"]

In [ ]:
# Action-specific info
all_action_info = [
    # Transfer
    {
        "name": "Transfer",
        "nospace_name": "Transfer",
        "action_url": "https://actions.globus.org/transfer/transfer",
        "action_scope": "https://auth.globus.org/scopes/actions.globus.org/transfer/transfer"
    },
    # Send Email
    {
        "name": "Send Email",
        "nospace_name": "SendEmail",
        "action_url": "https://actions.globus.org/notification/notify",
        "action_scope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/notification_notify"
    }
]

In [ ]:
# Deployments
selected_actions = [
    "Transfer",
    #"Send Email"
]
deploy_res = {}
for action_info in all_action_info:
    action_name = action_info["name"]
    if action_name not in selected_actions:
        continue

    # Can't .format() a JSON string, because objects {} are interpreted as
    # placeholders. Instead, .replace() #variable#.
    definition_str = json.dumps(loop_flow_def)
    for var_name, val in action_info.items():
        replace_str = "#" + var_name + "#"
        definition_str = definition_str.replace(replace_str, val)
    flow_definition = json.loads(definition_str)

    #print(json.dumps(flow_definition, indent=4, sort_keys=True))
    #print(flow_definition["definition"])

    flow_deploy_res = flows_client.deploy_flow(
        flow_definition=flow_definition["definition"],
        title=flow_definition["title"],
        description=flow_definition["description"],
        visible_to=flow_definition["visible_to"],
        runnable_by=flow_definition["runnable_by"],
    )

    deploy_res[action_name] = {
        "flow_id": flow_deploy_res["id"],
        "flow_scope": flow_deploy_res["globus_auth_scope"],
        "flow_url": flow_deploy_res["action_url"]
    }
    print(action_name + ":")
    print(deploy_res[action_name]["flow_id"])
    print(deploy_res[action_name]["flow_url"])
    print(deploy_res[action_name]["flow_scope"])
    print()

In [ ]:
# Test inputs
action_test_inputs = [{
    "name": "Transfer",
    "action_inputs": [{
        # jgaff_laptop to MDF Connect Dev-P
        "destination_endpoint_id": "ca7550ad-55a9-4762-b558-8f2b15049039",
        "label": "flow_test_a",  # optional
        "source_endpoint_id": "ce6d512a-b414-11e7-b0a7-22000a92523b",
        "transfer_items": [{
            "destination_path": "/home/ubuntu/data/deleteme/",
            "source_path": "/home/jgaff/Downloads/connect_demo/",
            "recursive": True
            }]
    }, {
        # MDF Connect Dev-P to NCSA
        "destination_endpoint_id": "82f1b5c6-6e9b-11e5-ba47-22000b92c6ec",
        "label": "flow_test_b",  # optional
        "source_endpoint_id": "ca7550ad-55a9-4762-b558-8f2b15049039",
        "transfer_items": [{
            "destination_path": "/test/deleteme_outcar",
            "source_path": "/home/ubuntu/data/deleteme/Ag2Al.hP2/OUTCAR",
            "recursive": False
        }]
    }, {
        "destination_endpoint_id": "e38ee745-6d04-11e5-ba46-22000b92c6ec",
    "label": "MDF Flow Test Transfer1",
    "source_endpoint_id": "e38ee745-6d04-11e5-ba46-22000b92c6ec",
    "transfer_items": [
      {
        "destination_path": "/MDF/mdf_connect/test_files/deleteme/data/test123/",
        "recursive": True,
        "source_path": "/MDF/mdf_connect/test_files/canonical_datasets/dft/"
      }
    ]
    }]}]
'''
}, {
    "name": "Send Email",
    "action_inputs": [{
        "body_template": "This is a test email for $name",
        "body_variables": {
            "name": "the MDF-developed Send Multiple Emails Flow"
        },
        "destination": "jgaff@uchicago.edu",
        #"destination": "materialsdatafacility@uchicago.edu",
        "send_credentials": [{
            # "credential_method": "",
            "credential_type": "smtp",
            "credential_value": {
                "hostname": smtp_hostname,
                "username": smtp_user,
                "password": smtp_pass
            }
        }],
        "sender": "materialsdatafacility@uchicago.edu",
        "subject": "[Ignore] Flow Email Testing",
        "__Private_Parameters": ["send_credentials"]
    }, {
        "body_template": "This is a different email, testing $text for $name",
        "body_variables": {
            "name": "the MDF-developed Send Multiple Emails Flow",
            "text": "multiple emails sent at once"
        },
        "destination": "jgaff@uchicago.edu",
        #"destination": "materialsdatafacility@uchicago.edu",
        # "notification_method": "",
        # "notification_priority": "low",
        "send_credentials": [{
            # "credential_method": "",
            #"credential_type": "ses",
            #"credential_value": {
            #    "aws_access_key_id": aws_id,
            #    "aws_secret_access_key": aws_secret,
            #    "region_name": aws_region
            #}
            "credential_type": "smtp",
            "credential_value": {
                "hostname": smtp_hostname,
                "username": smtp_user,
                "password": smtp_pass
            }
        }],
        "sender": "materialsdatafacility@uchicago.edu",
        "subject": "[Ignore] Flow Email Testing 2",
        "__Private_Parameters": ["send_credentials"]
    }]
}]
'''

In [ ]:
# Test submissions
selected_actions = [
    "Transfer",
    #"Send Email"
]
test_res = {}
for test_input in action_test_inputs:
    action_name = test_input["name"]
    if action_name not in selected_actions:
        continue
    elif action_name not in deploy_res.keys():
        print("Action '{}' not deployed".format(action_name))
        continue

    flow_info = deploy_res[action_name]
    flow_test_res = flows_client.run_flow(flow_info["flow_id"],
                                          flow_info["flow_scope"],
                                          test_input)
    test_res[action_name] = flow_test_res.data
    print(action_name, "Flow test submitted")
print(json.dumps(test_res, indent=4, sort_keys=True))

In [ ]:
# Check status of tests
for action_name, test_info in test_res.items():
    flow_info = deploy_res[action_name]
    status = flows_client.flow_action_status(flow_info["flow_id"],
                                             flow_info["flow_scope"],
                                             test_info["action_id"]).data
    print(action_name)
    print(flow_info["flow_id"], test_info["action_id"])
    print(json.dumps(status, indent=4, sort_keys=True))
    print()

In [ ]:
deploy_res

In [ ]:
action = "Send Email"
'''flows_client.flow_action_log(deploy_res[action]["flow_id"],
                             deploy_res[action]["flow_scope"],
                             test_res[action]["action_id"], limit=100).data'''

In [ ]:
aws_region = "us-east-1"
# Templates cannot contain newlines
curator_email_template = ("A new submission is available for curation. "
"Please visit this webpage to curate the submission: $landing_page "
"(If this link leads to a 404 Not Found page after one minute, it has already been curated.)")
author_email_template = ("Your submission has been $decision by a curator "
"for the following reason: $reason")
preamble_text = "Please review the submission and accept or reject it using the buttons below. You must also provide a reason for your decision."
text_box_prompt = "Please enter a reason for your decision. This will be stored and provided to the submitter."
flow_permissions = ["urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1"]  # jgaff@uchicago.edu


In [ ]:
curation_flow_def = {
    "definition": {
        "StartAt": "ChooseNotifyCurator",
        "States": {
            "ChooseNotifyCurator": {
                "Type": "Choice",
                "Choices": [{
                    "Variable": "$.curator_emails",
                    "BooleanEquals": False,
                    "Next": "CurateData"
                }],
                "Default": "EmailVarsStart"
            },
            "EmailVarsStart": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/expression_eval",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "expressions": [{
                        "expression": "[]",
                        "result_path": "email_inputs"
                    }, {
                        "expression": "dests",
                        "arguments": {
                            "dests.$": "$.curator_emails"
                        },
                        "result_path": "dests"
                    }]
                },
                "ResultPath": "$.inputs",
                "WaitTime": 86400,
                "Next": "EmailVarsLoop"
            },
            "EmailVarsLoop": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/expression_eval",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "expressions": [{
                        "expression": "current_inputs + [new_input]",
                        "arguments": {
                            "current_inputs.$": "$.inputs.details.email_inputs",
                            "new_input": {
                                # "body_mimetype": "",
                                "body_template.=": ("`$.curator_template` or '"
                                                    + curator_email_template + "'"),
                                "body_variables": {
                                    "landing_page.=": ("'https://actions.globus.org/weboption/landing_page/' "
                                                       "+ `$._context.action_id`")
                                },
                                "destination.=": "`$.inputs.details.dests`[0]",
                                # "notification_method": "",
                                # "notification_priority": "low",
                                "send_credentials.$": "$.send_credentials",
                                "sender.$": "$.email_sender",
                                "subject": "Curation Task Available",
                                "__Private_Parameters": ["send_credentials"]
                            }
                        },
                        "result_path": "email_inputs"
                    }, {
                        "expression": "dests[1:] if dests[1:] else False",
                        "arguments": {
                            "dests.$": "$.inputs.details.dests"
                        },
                        "result_path": "dests"
                    }]
                },
                "ResultPath": "$.inputs",
                "WaitTime": 86400,
                "Next": "EmailVarsChoice"
            },
            "EmailVarsChoice": {
                "Type": "Choice",
                "Choices": [{
                    "Variable": "$.inputs.details.dests",
                    "BooleanEquals": False,
                    "Next": "SendCuratorEmails"
                }],
                "Default": "EmailVarsLoop"
            },
            "SendCuratorEmails": {
                "Type": "Action",
                "ActionUrl": deploy_res["Send Email"]["flow_url"],
                "ActionScope": deploy_res["Send Email"]["flow_scope"],
                #"ExceptionOnActionFailure": True,
                "Parameters": {
                    "action_inputs.$": "$.inputs.details.email_inputs"
                },
                "ResultPath": "$.CuratorEmailsResult",
                "WaitTime": 86400,
                "Next": "CurateData"
            },
            "CurateData": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/weboption/wait_for_option",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/weboption_wait_for_option",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "landing_page": {
                        # "favicon_url": "",
                        # "header_background": "",
                        # "header_icon_link": "",
                        # "header_icon_url": "",
                        "display_options_as": "button",
                        "header_text.=": "'Curation for Submission ' + `$._context.action_id`",
                        # "page_title": "",
                        "preamble_text.=": ("`$.curation_text` or '"
                                            + preamble_text + "'"),
                        "selectable_by.$": "$.curation_permissions",
                        "include_text_input_form": True,
                        "text_input_form_prompt": text_box_prompt,
                        "url_suffix.$": "$._context.action_id"
                    },
                    "options": [{
                        "completed_message": "You have accepted this submission. Thank you!",
                        "description": "Accept this submission",
                        "name": "accepted",
                        "selectable_by.$": "$.curation_permissions",
                        "url_suffix.=": "`$._context.action_id` + '_accept'"
                    }, {
                        "completed_message": "You have rejected this submission. Thank you!",
                        "description": "Reject this submission",
                        "name": "rejected",
                        "selectable_by.$": "$.curation_permissions",
                        "url_suffix.=": "`$._context.action_id` + '_reject'"
                    }]
                },
                "ResultPath": "$.CurationResult",
                "WaitTime": 86400,
                "Next": "ChooseNotifyAuthor"
            },
            "ChooseNotifyAuthor": {
                "Type": "Choice",
                "Choices": [{
                    "Variable": "$.author_email",
                    "BooleanEquals": False,
                    "Next": "FinishCuration"
                }],
                "Default": "SendAuthorEmail"
            },
            "SendAuthorEmail": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/notification/notify",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/notification_notify",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    # "body_mimetype": "",
                    "body_template.=": ("`$.author_template` or '"
                                        + author_email_template + "'"),
                    "body_variables": {
                        "curation_task_id.$": "$._context.action_id",
                        "decision.$": "$.CurationResult.details.name",
                        "reason.$": "$.CurationResult.details.parameters.user_input"
                    },
                    "destination.$": "$.author_email",
                    # "notification_method": "",
                    # "notification_priority": "low",
                    "send_credentials.$": "$.send_credentials",
                    "sender.$": "$.email_sender",
                    "subject": "Submission Curated",
                    "__Private_Parameters": ["send_credentials"]
                },
                "ResultPath": "$.AuthorEmailResult",
                "WaitTime": 86400,
                "Next": "FinishCuration"
            },
            "FinishCuration": {
                "Type": "Pass",
                "End": True
            }
        }
    },
    "description": ("Run a curation flow with email notification. This Flow emails a curator, creates a "
                    "curation page accessible by the curator(s), and then emails the author the decision."),
    "runnable_by": flow_permissions,
    "synchronous": False,
    "title": "Curation with Notification",
    "types": ["Action", "Choice"],
    "visible_to": flow_permissions
}
input_schema = {
    "curator_emails": "list of str, or False",
    "curator_template": "str or False",  # variables: $landing_page
    "curation_permissions": "list of str",
    "curation_text": "str or False",
    "author_email": "str or False",
    "author_template": "str or False",  # variables: $curation_task_id, $decision, $reason
    "email_sender": "str",
    "send_credentials": [{}]
}

In [ ]:
'''curation_flow_deploy_res = flows_client.deploy_flow(
    flow_definition=curation_flow_def["definition"],
    title=curation_flow_def["title"],
    description=curation_flow_def["description"],
    visible_to=curation_flow_def["visible_to"],
    runnable_by=curation_flow_def["runnable_by"],
)
curation_flow_deploy_res'''

In [ ]:
#curation_flow_id = curation_flow_deploy_res["id"]
#curation_flow_scope = curation_flow_deploy_res["globus_auth_scope"]
curation_flow_id = "a24d39dd-f8b7-4287-ba4d-cdd8e36fcee6"
curation_flow_scope = "https://auth.globus.org/scopes/a24d39dd-f8b7-4287-ba4d-cdd8e36fcee6/flow_a24d39dd_f8b7_4287_ba4d_cdd8e36fcee6"
print(curation_flow_id)
print(curation_flow_deploy_res["action_url"])
print(curation_flow_scope)

In [ ]:
curation_flow_input = {
    #"curator_emails": ["jgaff@uchicago.edu", "jgaff@globus.org"],
    "curator_emails": ["jgaff@uchicago.edu"],
    "curator_template": "Curator Template test: $landing_page",
    "curation_text": "Here is some sample text. https://example.org",
    "author_email": "jgaff@uchicago.edu",
    "author_template": "This is a test email for '$curation_task_id': Decision: $decision, reason: $reason",
    "email_sender": "materialsdatafacility@uchicago.edu",
    "curation_permissions": ["public"],  #["urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1"],
    "send_credentials": [{
        # "credential_method": "",
        "credential_type": "smtp",
        "credential_value": {
            "hostname": smtp_hostname,
            "username": smtp_user,
            "password": smtp_pass
#        "credential_type": "ses",
#        "credential_value": {
#            "aws_access_key_id": aws_id,
#            "_private_aws_secret_access_key": aws_secret,
#            "region_name": aws_region
        }
    }],
    "__Private_Parameters": ["send_credentials"]
}

In [ ]:
curation_flow_res = flows_client.run_flow(curation_flow_id,
                                          curation_flow_scope,
                                          curation_flow_input)
curation_flow_res.data

In [ ]:
flows_client.flow_action_status(curation_flow_id,
                                curation_flow_scope,
                                curation_flow_res["action_id"]).data

In [ ]:
flows_client.flow_action_log(curation_flow_id,
                             curation_flow_scope,
                             curation_flow_res["action_id"], limit=100).data

In [ ]:
send_credentials = [{
                        # "credential_method": "",
                        "credential_type": "smtp",
                        "credential_value": {
                            "hostname": smtp_hostname,
                            "username": smtp_user,
                            "password": smtp_pass
                        }
                    }]

In [ ]:
dumb_flow_def = {
    "StartAt": "EmailVarsStart",
    "States": {
        "EmailVarsStart": {
            "Type": "Action",
            "ActionUrl": "https://actions.globus.org/expression_eval",
            "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
            "ExceptionOnActionFailure": True,
            "Parameters": {
                "expressions": [{
                    "expression": "[]",
                    "result_path": "email_inputs"
                }, {
                    "expression": "dests",
                    "arguments": {
                        "dests": ["jgaff@uchicago.edu", "jgaff@globus.org"]
                    },
                    "result_path": "dests"
                }]
            },
            "ResultPath": "$.inputs",
            "WaitTime": 86400,
            "Next": "EmailVarsLoop"
        },
        "EmailVarsLoop": {
            "Type": "Action",
            "ActionUrl": "https://actions.globus.org/expression_eval",
            "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
            "ExceptionOnActionFailure": True,
            "Parameters": {
                "expressions": [{
                    "expression": "current_inputs + [new_input]",
                    "arguments": {
                        "current_inputs.$": "$.inputs.details.email_inputs",
                        "new_input": {
                            # "body_mimetype": "",
                            "body_template.=": ("`$._context.action_id` + '"
                                                + curator_email_template + "'"),
                            "body_variables": {
                                "landing_page.=": ("'https://actions.globus.org/weboption/landing_page/' "
                                                   "+ `$._context.action_id`")
                            },
                            "destination.=": "`$.inputs.details.dests`[0]",
                            # "notification_method": "",
                            # "notification_priority": "low",
                            "send_credentials": send_credentials,
                            "sender": "materialsdatafacility@uchicago.edu",
                            "subject": "Curation Task Available",
                            "__Private_Parameters": ["send_credentials"]
                        }
                    },
                    "result_path": "email_inputs"
                }, {
                    "expression": "dests[1:] if dests[1:] else False",
                    "arguments": {
                        "dests.$": "$.inputs.details.dests"
                    },
                    "result_path": "dests"
                }]
            },
            "ResultPath": "$.inputs",
            "WaitTime": 86400,
            "Next": "EmailVarsChoice"
        },
        "EmailVarsChoice": {
            "Type": "Choice",
            "Choices": [{
                "Variable": "$.inputs.details.dests",
                "BooleanEquals": False,
                "Next": "EndStep"
            }],
            "Default": "EmailVarsLoop"
        },
        "EndStep": {
            "Type": "Action",
            "ActionUrl": "https://actions.globus.org/weboption/wait_for_option",
            "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/weboption_wait_for_option",
            "ExceptionOnActionFailure": True,
            "Parameters": {
                "landing_page": {
                    # "favicon_url": "",
                    # "header_background": "",
                    # "header_icon_link": "",
                    # "header_icon_url": "",
                    "display_options_as": "button",
                    "header_text.=": "'Curation for Submission ' + `$._context.action_id`",
                    # "page_title": "",
                    "preamble_text": preamble_text,
                    "selectable_by": ["public"],
                    "include_text_input_form": True,
                    "text_input_form_prompt": text_box_prompt,
                    "url_suffix.$": "$._context.action_id"
                },
                "options": [{
                    "completed_message": "You have accepted this submission. Thank you!",
                    "description": "Accept this submission",
                    "name": "accepted",
                    "selectable_by": ["public"],
                    "url_suffix.=": "`$._context.action_id` + '_accept'"
                }, {
                    "completed_message": "You have rejected this submission. Thank you!",
                    "description": "Reject this submission",
                    "name": "rejected",
                    "selectable_by": ["public"],
                    "url_suffix.=": "`$._context.action_id` + '_reject'"
                }]
            },
            "ResultPath": "$.CurationResult",
            "WaitTime": 86400,
            "End": True
        }
    }
}

In [ ]:
'''
dumb_flow_deploy = flows_client.deploy_flow(
    flow_definition=dumb_flow_def,
    title="Testing Flow",
    visible_to=["urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1"],
    runnable_by=["urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1"],
)
dumb_flow_deploy
'''

In [ ]:
dumb_flow_id = dumb_flow_deploy["id"]
dumb_flow_scope = dumb_flow_deploy["globus_auth_scope"]
print(dumb_flow_id)
print(dumb_flow_scope)

In [ ]:
dumb_flow_input = {
    # pass
}

In [ ]:
dumb_flow_res = flows_client.run_flow(dumb_flow_id,
                                      dumb_flow_scope,
                                      dumb_flow_input)
dumb_flow_res.data

In [ ]:
flows_client.flow_action_status(dumb_flow_id,
                                dumb_flow_scope,
                                dumb_flow_res["action_id"]).data

In [ ]:
flow_permissions

In [ ]:
ref_auth = list(mdf_toolbox.login(services="https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/notification_notify").values())[0]
action_client = globus_automate_client.create_action_client("https://actions.globus.org/notification/notify", ref_auth.access_token)


In [ ]:
email_template = """A new dataset has been submitted to MDF Connect.
Title: $title
Submitter: $submitter ($email)
Source ID: $source_id
Time submitted: $sub_time
Curation required: $curation
"""
email_template2 = "Test email\nTitle: $title\nSubmitter: $submitter ($email)\n\nEnd"
email_creds = {
    "credential_type": "smtp",
        "credential_value": {
            "hostname": smtp_hostname,
            "username": smtp_user,
            "password": smtp_pass
        }
}
body = {
    # "body_mimetype": "",
    "body_template": email_template2,
    "body_variables": {
        "title": "Email Test",
        "submitter": "JGaff",
        "email": "foo@bar.com",
        "source_id": "foobar_v99.99",
        "sub_time": "26:98PM",
        "curation": str(True)
    },
    #"destination": "materialsdatafacility@uchicago.edu",
    "destination": "jgaff@uchicago.edu",
    "send_credentials": [email_creds],
    "sender": "materialsdatafacility@uchicago.edu",
    "subject": "New MDF Connect Submission",
    #"__Private_Parameters": ["send_credentials"]
}

In [ ]:
res = action_client.run(body)

In [ ]:
res

In [ ]:
action_client.status(res["action_id"]).data

In [ ]:
from getpass import getpass
dc_username = getpass("Datacite Username: ")
dc_password = getpass("Datacite Password: ")

In [ ]:
#action_url = "http://siap-elb-sandbox-970327309.us-east-1.elb.amazonaws.com/"
#action_scope = "https://auth.globus.org/scopes/a9b4124f-887a-461e-ba72-fa8ea701a8f2/siap_ingest_scope"
#action_url = "https://actions.globus.org/search/ingest"
#action_url = "https://actions.globus.org/datacite/mint/basic_auth"
action_url = "https://actions.globus.org/datacite/mint/dc_schema"
action_scope = None

action_client2 = globus_automate_client.create_action_client(action_url, action_scope)

In [ ]:
#print(json.dumps(action_client2.introspect().data, indent=4, sort_keys=True))
action_client2.introspect()

In [ ]:
"""
# SIAP
action_input = {
    #"auth_header": "",
    "index": mdf_toolbox.translate_index("mdf-dev"),
    "locations": ["https://317ec.36fe.dn.glob.us/public/mock_feedstock.json"],
    "require_all_success": True
}
action_input
"""
"""
# Single-entry SIAP
action_input = {
    "subject": "https://example.com/foobar",
    "visible_to": ["public"],
    "content": {
        "dev_test": True
    },
    "search_index": mdf_toolbox.translate_index("mdf-dev")
}
"""
# DOI Mint
"""
dc_data = {'data': {'type': 'dois',
  'attributes': {'titles': [{'title': 'Base Deploy Testing Dataset'}],
   'creators': [{'familyName': '',
     'givenName': 'jgaff',
     'name': 'jgaff',
     'affiliation': ['UChicago']}],
   'publisher': 'Materials Data Facility',
   'publicationYear': '2020',
   'url': 'https://example.com/',
   'doi': '10.23677/gl-is-ku-xb-pa',
   'identifiers': [{'identifier': '10.23677/gl-is-ku-xb-pa',
     'identifierType': 'DOI'}],
   'types': {'resourceTypeGeneral': 'Dataset', 'resourceType': 'Dataset'},
   'event': 'publish'}}}
action_input = {
    "Doi": dc_data,
    "as_test": True,
    "username": dc_username,
    "password": dc_password
}
action_input = {
    "as_test": True,
    "username": dc_username,
    "password": dc_password,
    "Doi": {
        "id": "10.23677/mdf_ap_test2",
        "type": "dois",
        "attributes": {
            "doi": "10.23677/mdf_ap_test2",
            "creators": [
                {
                    "name": "Globus Dev Team"
                }
            ],
            "titles": [
                {
                    "title": "Test Title"
                }
            ],
            "publisher": "Globus",
            "publicationYear": "2020"
        }
    }
}
"""
dc_block = {
    "identifier": {"identifier": "10.23677/mdf_ap_test3", "identifierType": "DOI"},
 'titles': [{'title': 'Base Deploy Testing Dataset'}],
 'creators': [{'creatorName': 'jgaff',
   'familyName': '',
   'givenName': 'jgaff',
   'affiliations': ['UChicago']}],
 'publisher': 'Materials Data Facility',
 'publicationYear': '2020',
 'resourceType': {'resourceTypeGeneral': 'Dataset', 'resourceType': 'Dataset'}}

action_input = {
    "as_test": True,
    #"doi": "10.23677/mdf_ap_test3",
    "username": dc_username,
    "password": dc_password,
    "Doi": {"dc": dc_block}
}

jgaff = "urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1"

In [ ]:
res = action_client2.run(body=action_input, monitor_by=[jgaff], manage_by=[jgaff])

In [ ]:
#res

In [ ]:
action_client2.status(res["action_id"]).data

In [ ]:
action_url = "https://actions.globus.org/transfer/transfer"

action_client3 = globus_automate_client.create_action_client(action_url)

In [ ]:
action_input = {
        "destination_endpoint_id": "e38ee745-6d04-11e5-ba46-22000b92c6ec",
        "label": "MDF Flow Test Transfer1",
        "source_endpoint_id": "e38ee745-6d04-11e5-ba46-22000b92c6ec",
        "transfer_items": [
            {
                "destination_path": "/MDF/mdf_connect/test_files/deleteme/data/test123/",
                "recursive": True,
                "source_path": "/MDF/mdf_connect/test_files/canonical_datasets/dft/"
            }
        ]
    }

In [ ]:
res = action_client3.run(action_input)

In [ ]:
res

In [ ]:
status = action_client3.status(res["action_id"]).data
print(json.dumps(status, indent=4, sort_keys=True))